In [ ]:
# Step 1: Install necessary libraries
!pip install pandas matplotlib boto3 datasets evaluate transformers[sentencepiece] accelerate seqeval seaborn mlflow optimum[openvino,nncf] psutil pynvml -q

## Dataset manipulation & env preparation

In [ ]:
### Load all pipeline parameters
import os
key_id = os.environ.get('S3_ACCESS_KEY_ID')
secret_key = os.environ.get('S3_SECRET_ACCESS_KEY')
model_name= os.environ.get("model_name")
bucket_name= os.environ.get("bucket")
run_name=os.environ.get("mlflow_run_name")

In [ ]:
#Define your input column and your output column
input_column_name="text"
output_column_name="sentiment"

In [ ]:
import pandas as pd

labeled_dataset = "dataset.csv"

# Assuming the file is in the current working directory
df = pd.read_csv(labeled_dataset)

# Display the first few rows of the dataframe
df.head()

## Give a name to your model and version  🧙‍♂️🧙‍♂️

This process is crucial mainly because a `text-classification` model can be intended for a huge amount of approaches

In [ ]:
import os
experiment = os.environ.get("mlflow_experiment")
base_model_name = os.environ.get("base_model")
run_name = os.environ.get("mlflow_run_name")
test_size = float(os.environ.get("test_size"))

### Model manipulation 

In [ ]:
import json

# Opening JSON file
file_label2id = open('label2id.json')
file_id2label = open('id2label.json')
label2id = json.load(file_label2id)
id2label=json.load(file_id2label)
print(f"The label2id json loaded correctly: {label2id}")
print(f"The id2label json loaded correctly: {id2label}")
os.remove("label2id.json")
os.remove("id2label.json")

In [ ]:
# Add the new 'label' column to the dataframe by mapping values from the 'category' column
df['label'] = df[output_column_name].replace(label2id)
df.head(3)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=test_size)

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples[input_column_name], truncation=True)

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=len(label2id),id2label=id2label, label2id=label2id)

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    conf_matrix = confusion_matrix(labels, predictions)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    
    # Save the confusion matrix plot
    conf_matrix_filepath = "confusion_matrix.png"
    plt.savefig(conf_matrix_filepath)
    plt.close()
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
#Mlflow
import mlflow

os.environ["MFLOW_EXPERIMENT_NAME"]=experiment

In [ ]:
## hyperparameters 
lr=float(os.environ.get("learning_rate"))
train_batch_size=int(os.environ.get("batch_size"))
eval_batch_size=int(os.environ.get("batch_size"))
epochs=int(os.environ.get("epochs"))
decay=float(os.environ.get("decay"))
eval_strategy="epoch"
log_strategy="epoch"

In [ ]:
training_args = TrainingArguments(
    hub_model_id=model_name,
    output_dir=run_name,
    learning_rate=lr,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    num_train_epochs=epochs,
    weight_decay=decay,
    evaluation_strategy=eval_strategy,
    logging_strategy=log_strategy,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_dataset=tokenized_test
)

In [ ]:
import mlflow

mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))
mlflow.set_experiment(experiment)
mlflow.enable_system_metrics_logging()

In [ ]:
### Fine tune model
trainer.train()

In [ ]:
## Save pytorch 
trainer.save_model(model_name)

In [ ]:
experiment = mlflow.get_experiment_by_name(experiment)
filter_string = f"tags.mlflow.runName = '{run_name}'"
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string=filter_string
)

# Extract the run_id from the DataFrame
if not runs.empty:
    previous_run_id = runs.iloc[0]['run_id']
    print(f"Run ID: {previous_run_id}")
else:
    print("No run found with the specified name.")

In [ ]:
import mlflow.data
from mlflow.data.pandas_dataset import PandasDataset
from optimum.onnxruntime import ORTModelForSequenceClassification
from pathlib import Path
import onnx

In [ ]:
def uploadModel(run_id:str):
    train_dataset: PandasDataset = mlflow.data.from_pandas(df_train, source="Label Studio")
    test_dataset: PandasDataset = mlflow.data.from_pandas(df_test, source="Label Studio")
    with mlflow.start_run(run_id=previous_run_id) as run:
        ORTModelForSequenceClassification.from_pretrained(model_name,export=True).save_pretrained(f"{model_name}_onnx")
        tmp_dir = Path(f"{model_name}_onnx")
        mlflow.log_artifacts(tmp_dir, artifact_path=model_name)
        mlflow.log_artifact("confusion_matrix.png",artifact_path=model_name)
        tokenizer.save_pretrained(f"{model_name}_onnx")
        onnx_model = onnx.load_model(f"{model_name}_onnx/model.onnx")
        model_info = mlflow.onnx.log_model(onnx_model,model_name,registered_model_name=model_name)
        mlflow.log_input(train_dataset, context="training")
        mlflow.log_input(test_dataset,context="validation")
        mlflow.end_run()

In [ ]:
uploadModel(previous_run_id)

In [ ]:
mlflow.end_run()